# Creating the web scraper

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

# Getting the dataset

In [5]:
data = pd.read_csv('../data/music-subset.csv')

In [6]:
refined_data = data.loc[(data["rank"] == 1)]

In [7]:
# new_index = [1,2,3,4,5,6,7,8,9,10,11,12,133,14,15,16,17,18,19,20]

In [8]:
new_data = refined_data.reset_index(drop = True)

In [9]:
refined_data.shape

(1000, 20)

In [10]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

KeyError: 'trend'

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,195840.0
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,259196.0
2,2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,222560.0
3,3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,205600.0
4,4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,234320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Tim Tim - Ao Vivo,126,2017-01-01,Wesley Safadão,https://open.spotify.com/track/4bftZ7IayYg0I6q...,Brazil,top200,MOVE_UP,37363.0,...,2.0,-4.964,0.0,0.0504,0.58100,0.000000,0.1430,0.891,163.998,174067.0
996,996,Folgado - Ao Vivo,127,2017-01-01,Marília Mendonça,https://open.spotify.com/track/4DYmJ19DhmxUe26...,Brazil,top200,MOVE_UP,37320.0,...,1.0,-3.792,0.0,0.0729,0.64500,0.000000,0.0726,0.569,163.848,163902.0
997,997,Photograph,128,2017-01-01,Ed Sheeran,https://open.spotify.com/track/6fxVffaTuwjgEk5...,Brazil,top200,MOVE_DOWN,37266.0,...,4.0,-10.480,1.0,0.0476,0.60700,0.000464,0.0986,0.201,107.989,258987.0
998,998,Starving,129,2017-01-01,"Hailee Steinfeld, Grey, Zedd",https://open.spotify.com/track/1qOLh0tI7trd1zd...,Brazil,top200,MOVE_DOWN,37178.0,...,4.0,-4.230,1.0,0.1260,0.37100,0.000000,0.1020,0.512,99.721,181880.0


In [14]:
new_data.drop(index_names, inplace=True)

In [11]:
new_data.shape

(1000, 20)

In [30]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0
25,Rockabye (feat. Sean Paul & Anne-Marie),1,2017-01-01,Clean Bandit,https://open.spotify.com/track/5knuzwU65gJK7IF...,Hungary,top200,MOVE_UP,6263.0
...,...,...,...,...,...,...,...,...,...
227143,Diamond in the rough,1,2021-07-30,Arik Ancelin,https://open.spotify.com/track/54AtiA08i3sDfJY...,Iceland,viral50,MOVE_UP,NaN
227144,Aadmi Chutiya Hai,1,2021-07-30,Rahgir,https://open.spotify.com/track/2PEJKORCdmceTW2...,India,viral50,MOVE_UP,NaN
227196,Bezos I,1,2021-07-31,Bo Burnham,https://open.spotify.com/track/0Kdj7nwaYQmvhxn...,Czech Republic,viral50,MOVE_UP,NaN
227202,Paar tundi veel,1,2021-07-31,Shanon,https://open.spotify.com/track/3LM9EDiw6CtKhCt...,Estonia,viral50,MOVE_UP,NaN


In [17]:
contain_values = data[data.artist.str.contains('The Weeknd')]
print (contain_values)

     index                 title  rank        date  \
28      28               Starboy    29  2017-01-01   
75      75      I Feel It Coming    76  2017-01-01   
201    201               Starboy     1  2017-01-01   
207    207      I Feel It Coming     7  2017-01-01   
266    266         Party Monster    66  2017-01-01   
307    307               Rockin’   107  2017-01-01   
331    331    Can't Feel My Face   130  2017-01-01   
333    333             Sidewalks   132  2017-01-01   
368    368  Starboy - Kygo Remix   167  2017-01-01   
388    388        Six Feet Under   187  2017-01-01   
394    394             The Hills   193  2017-01-01   
408    408               Starboy     7  2017-01-01   
449    449      I Feel It Coming    47  2017-01-01   
560    560         Party Monster   158  2017-01-01   
605    605               Starboy     3  2017-01-01   
621    621      I Feel It Coming    19  2017-01-01   
725    725         Party Monster   123  2017-01-01   
819    819               Sta

In [13]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [18]:
new_data.head()

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


In [15]:
#new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [19]:
new_data.shape

(3, 22)

In [17]:
new_data.iloc[140087:140100,:]

,title,rank,date,artist,url,region,chart,trend,streams


In [18]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0


# Import dataset

In [12]:
data = pd.read_csv('../data/music-subset.csv', index_col=0)

In [14]:
data.head(10)

,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
index,,,,,,,,,,,,,,,,,,,
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0,0.0776,0.1870,0.000030,0.1590,0.9070,102.034,195840.0
1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0
2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0
3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.0,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0
4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,1.0,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0
5,7 rings,1,2019-03-16,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,Indonesia,125837.0,0.778,0.317,1.0,-10.732,0.0,0.3340,0.5920,0.000000,0.0881,0.3270,140.048,178627.0
6,Shape of You,1,2017-03-08,Ed Sheeran,https://open.spotify.com/track/7qiZfU4dY1lWllz...,Portugal,55132.0,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.9310,95.977,233713.0
7,China,1,2019-08-05,"Anuel AA, Daddy Yankee, KAROL G, J Balvin, Ozuna",https://open.spotify.com/track/2ksOAxtIxY8yElE...,Chile,579047.0,0.786,0.808,7.0,-3.702,1.0,0.0882,0.0846,0.000289,0.0822,0.6090,105.027,301714.0
8,Hawái,1,2020-09-20,Maluma,https://open.spotify.com/track/1yoMvmasuxZfqHE...,Spain,428210.0,0.783,0.727,6.0,-3.454,0.0,0.3890,0.1850,0.000025,0.0626,0.5580,180.067,199112.0


# Extracting artist and track name

In [36]:
track_nr = 0

In [18]:
import re

def get_track_name(nr):
    

    track_string = str(data.title[nr])
    
    x = track_string.find(" (")
    y = track_string.find(" -")


    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [37]:
get_track_name(track_nr)

'Chantaje'

In [38]:
get_track_artist(track_nr)

'Shakira'

# Getting URL

In [21]:
import re

def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', track)
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', artist)
    
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [39]:
get_url(track_nr)

'https://genius.com/Shakira-Chantaje-lyrics'

# Scraping credits

In [23]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')


        featur = []
        writer = []
        label = []
        producer = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')


            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    featur.append(x)



            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)



            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":featur, "Writer":writer, "Label":label, "Producer":producer}        
        
        
    return collection

In [44]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
#extLis = extract_credits(get_url(98))
extLis = extract_credits(get_url(track_nr))
extLis

{'Feature': ['Maluma'],
 'Writer': ['Kevin ADG', 'Chan El Genio', 'Kenai', 'Maluma', 'Shakira'],
 'Label': [],
 'Producer': ['The Rudeboyz', 'Shakira', 'Maluma']}

# Dict to csv

In [45]:
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

### Path to neo4j import folder
Nicholas Stationær: C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv

Martins stationær: C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-e0579ea2-8404-4255-955c-aa57a88
28e4f\import\collaborators.csv

In [46]:
df = pd.DataFrame.from_dict(credit_list)
df.to_csv (r'C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-e0579ea2-8404-4255-955c-aa57a8828e4f\import\collaborators.csv', index = False, header=True)

In [47]:
df

,Feature,Writer,Label,Producer
0,Maluma,Kevin ADG,None,The Rudeboyz
1,None,Chan El Genio,None,Shakira
2,None,Kenai,None,Maluma
3,None,Maluma,None,None
4,None,Shakira,None,None


***

# Neo4j

In [48]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [49]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test1234")


In [50]:
conn.query('''MATCH (n) RETURN n;''')

[]

In [53]:
artist = get_track_artist(track_nr)
track = get_track_name(track_nr)
track

'Chantaje'

In [54]:
artist

'Shakira'

### Query to insert into neo4j

In [55]:
def run_querys(artist, track):
    
    conn.query('''CALL apoc.load.csv('collaborators.csv')
    YIELD lineNo, map, list
    
    WHERE map.Feature IS NOT NULL
    MERGE (f:Person {name:map.Feature})
    MERGE (a:Person {name:"'''+ artist + '''"})
    MERGE (w:Person {name:map.Writer})
    MERGE (p:Person {name:map.Producer})
    MERGE (l:Label {name:map.Label})
    MERGE (s:Song {name:"'''+ track +'''"})
    
    
    MERGE (f)-[:COLLABORATED]-(a)
    MERGE (f)-[:FEATURED]->(s)
    MERGE (a)-[:SANG]->(s)
    MERGE (w)-[:WROTE]->(s)
    MERGE (l)-[:RELEASED]->(s)
    MERGE (p)-[:PRODUCED]->(s);
    ''')
    
    conn.query('''MATCH (f:Person) WHERE (f.name = "") detach delete f;''')
    
    conn.query('''MATCH (l:Label) WHERE (l.name = "") detach delete l;''')
    
    conn.close()

In [57]:
run_querys(artist, track)

In [58]:
df

,Feature,Writer,Label,Producer
0,Maluma,Kevin ADG,None,The Rudeboyz
1,None,Chan El Genio,None,Shakira
2,None,Kenai,None,Maluma
3,None,Maluma,None,None
4,None,Shakira,None,None


### Getting the Shit

In [2]:

track_nr = 449
# For scraping
extLis = extract_credits(get_url(track_nr))
# Get credit
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
# Running neo4j query
run_querys(track_nr)

NameError: name 'extract_credits' is not defined

In [73]:
import time
tic = time.perf_counter()

for index in data.index:
    tac = time.perf_counter()
    print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + f'{tac - tic:0.0f} seconds elapsed', end='\r')
    artist = get_track_artist(index)
    track = get_track_name(index)
    # For scraping
    extLis = extract_credits(get_url(track_nr))
    # Get credit
    credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
    # Running neo4j query
    run_querys(artist, track)
    tac = time.perf_counter()
toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

Query failed! {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '想': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 10, column 40 (offset: 337))
"    MERGE (s:Song {name:"想見你想見你想見你(電視劇"想見你"片尾曲)"})"
                                        ^}
Added new rows from external links in 2423.1876 secondselapsed
